# DLT Project Report
### Model comparison for LSTM and Feedfoward with timseries data set
###### 성균관대 통계학과 박나린 (석사) 2022711906

## 0. Setting

케라스 기반의 딥러닝 모델들을 쓰기 위한 기본 환경 설정을 해준다. 또한 파이썬 기반의 모형들을 쓰기 때문에, 파이썬에서 처리를 위한 패키지들도 불러 와준다.

In [1]:
import pandas as pd
import numpy as np
import bz2
import matplotlib.pyplot as plt
import sklearn

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

%matplotlib inline

In [2]:
import os
for dirname, _, filenames in os.walk('C:/Users/annie/Dropbox/data/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/annie/Dropbox/data/input\test.ft.txt.bz2
C:/Users/annie/Dropbox/data/input\train.ft.txt.bz2


## 1. Data Processing

이 데이터셋은 fastText를 학습하기 위한 아마존 고객 리뷰 몇 백만 개(입력 텍스트)와 별점(출력 레이블)으로 구성되어 있다. 
이 데이터셋은 양질의 비즈니스 데이터로 실제 규모이지만, 소규모 노트북에서 몇 분 안에 학습할 수 있도록 구성되었다. train 과 test 파일이 각각 존재 하므로 train test를 나눌 필요가 없다.

### 1.1 Import Data
먼저 bz2 파일 형식으로 되어있는 train, test 파일을 불러온다

In [3]:
# bz2 데이터 읽어 오는 def 만들어주기

def labels_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts

In [4]:
train_label, train_text = labels_texts('C:/Users/annie/Dropbox/data/input/train.ft.txt.bz2')
test_label, test_text = labels_texts('C:/Users/annie/Dropbox/data/input/test.ft.txt.bz2')

In [5]:
print(train_label[0])
print(train_text[0])

1
Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^



텍스트를 처리하기 위해 첫 번째로 수행할 작업은 모든 문자를 소문자로 변환하고, 단어가 아닌 문자를 제거 하는 것이다. 대부분의 경우 이는 구두점으로 대체될 것이다. 그런 다음, 강세 기호가 있는 문자와 같은 다른 문자를 제거한다.. 이러한 문자들 중 일부는 정규 ASCII 문자로 대체하는 것이 더 좋을 수도 있지만, 여기서는 그것을 무시하기로 결정했다. 또한, 이 코퍼스에서는 다양한 문자의 빈도를 살펴보면 매우 적은 수의 특이한 문자가 있는 것으로 나타났다.

In [6]:
import re

not_numChar = re.compile(r'[\W]')
no_encode = re.compile(r'[^a-z0-1\s]')

def normalisation(texts):
    norm_text = []
    for word in texts:
        lower = word.lower()
        not_punct = not_numChar.sub(r' ', lower)
        exclude_no_encode = no_encode.sub(r'', not_punct)
        norm_text.append(exclude_no_encode)
    return norm_text

In [7]:
train_text = normalisation(train_text)
test_text = normalisation(test_text)

In [8]:
print(train_text[0])

stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen    


In [9]:
y_train = np.array(train_label)
y_test = np.array(test_label)
print(y_train.shape)
print(y_test.shape)

(3600000,)
(400000,)


### 1.2 Tokenize the data

토크나이즈(tokenize)는 텍스트를 작은 단위로 분할하는 과정으로 토크나이즈를 통해 텍스트를 단어 단위로 분할하면 자연어 처리 작업에 대해 더욱 정확한 분석이 가능해진다. 각각의 단어는 개별적인 의미를 가지며, 이를 활용하여 문장 또는 문서의 의미를 파악하거나 감성 분석과 같은 작업을 수행할 수 있는 구조를 뛰게 해준다.


In [10]:
max_features = 8192 # We will only consider the top 10,000 words in the dataset
maxlen = 128  # We will cut reviews after 128 words
embed_size = 64  

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_text)

In [11]:
training_token = tokenizer.texts_to_sequences(train_text)
testing_token = tokenizer.texts_to_sequences(test_text)

In [12]:
x_train = pad_sequences(training_token, maxlen = maxlen, padding = 'post')
x_test = pad_sequences(testing_token, maxlen = maxlen, padding = 'post')

# 2. Model Fitting
## 2.1 CNN


CNN은 "Convolutional Neural Network"의 약자로, 합성곱 신경망이라고도 불린다. 이는 주로 이미지 및 시계열 데이터와 같은 그리드 형식의 입력에 사용되는 신경망 아키텍처로 특히 이미지 인식, 컴퓨터 비전 및 자연어 처리와 관련된 작업에 많이 사용된다.

CNN은 입력 데이터의 지역적인 패턴 및 구조를 인식하기 위해 컨볼루션 연산을 사용하는데 이는 입력 데이터를 작은 윈도우로 나누고, 각 윈도우에 대해 필터와의 컨볼루션을 계산하는 것을 의미한다. 이러한 컨볼루션 연산은 데이터의 특징을 추출하는 데 도움이 된다. 컨볼루션 연산의 결과는 활성화 함수를 통과한 뒤, 풀링(pooling) 레이어를 통해 다운샘플링 되고, 이 과정을 반복하여 신경망은 점차 더 추상적인 수준의 특징을 학습하게 되는 것이다.

CNN은 이러한 컨볼루션 및 풀링 레이어의 조합으로 구성되며, 일반적으로 여러 개의 컨볼루션 레이어와 풀링 레이어가 번갈아가며 쌓이는 형태를 가지고 있다. 이후에는 전체 특징 맵을 완전 연결(fully connected) 레이어로 연결하여 최종 분류를 수행하게 된다.

CNN은 이미지 처리 작업에서 좋은 성능을 보이며, 최근에는 자연어 처리 작업에도 적용되고 있다. 텍스트 분류, 감성 분석, 문장 생성 등의 자연어 처리 작업에서 CNN은 텍스트의 지역적인 구조와 패턴을 학습하여 효과적인 모델을 구축하는 데 도움이 된다.

따라서 먼저 CNN구조의 신경망을 사용하여 모델 적합을 해보기로 하였다.

In [13]:
def build_model():
    sequences = layers.Input(shape=(maxlen,))
    embedded = layers.Embedding(max_features, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
model = build_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding (Embedding)       (None, 128, 64)           524288    
                                                                 
 conv1d (Conv1D)             (None, 126, 64)           12352     
                                                                 
 batch_normalization (BatchN  (None, 126, 64)          256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 42, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 64)            20544 

In [14]:
history=model.fit( x_train, y_train, batch_size=64, epochs=2, validation_data=(x_test, y_test),)

Epoch 1/2
56250/56250 [==============================] - 2710s 48ms/step - loss: 0.1860 - binary_accuracy: 0.9286 - val_loss: 0.1660 - val_binary_accuracy: 0.9378
Epoch 2/2
56250/56250 [==============================] - 2557s 45ms/step - loss: 0.1638 - binary_accuracy: 0.9388 - val_loss: 0.1649 - val_binary_accuracy: 0.9383


In [15]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

KeyError: 'acc'

In [ ]:
preds = model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

## 2.2 RNN

RNN은 "Recurrent Neural Network"의 약자로, 순환 신경망이라고도 불린다. 이는 주로 시퀀스 데이터를 처리하는 데 사용되는 신경망 아키텍로, 이전 단계에서 계산된 결과를 현재 단계의 입력으로 반복적으로 전달하여 순환적인 구조를 형성한다.

RNN은 시퀀스 데이터의 특성을 이해하기 위해 시간적인 정보를 고려하여 각 단계에서 RNN은 현재 입력과 이전 단계의 은닉 상태를 함께 출력을 계산한다. 이전 단계의 정보가 현재 단계로 전달되기 때문에 RNN은 시퀀스의 장기 의존성을 학습할 수 있고 이러한 특성은 자연어 처리와 음성 인식과 같은 작업에 유용하게 된다.

RNN은 기본적으로 동일한 가중치를 모든 시간 단계에서 재사용하며, 재귀적인 구조를 가지고 있기 때문에 오래된 정보를 유지하면서 새로운 입력에 대한 정보를 반영할 수 있다. 그러나 RNN에는 "장기 의존성 문제"가 있어, 긴 시퀀스에서는 이전 정보를 제대로 기억하지 못하는 경우가 발생할 수 있다.

이러한 문제를 해결하기 위해 LSTM(Long Short-Term Memory)과 GRU(Gated Recurrent Unit)와 같은 RNN의 변형 모델이 개발되었고. LSTM과 GRU는 RNN의 기본 구조를 확장하여 장기 의존성을 더 잘 학습할 수 있도록 도와준다

In [ ]:
def build_rnn_model():
    sequences = layers.Input(shape=(maxlen,))
    embedded = layers.Embedding(max_features, 64)(sequences)
    x = layers.RNN(128, return_sequences=True)(embedded)
    x = layers.RNN(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model2 = models.Model(inputs=sequences, outputs=predictions)
    model2.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model2
    
rnn_model = build_rnn_model()
model.summary()

In [ ]:
history=rnn_model.fit(train_texts, train_labels,  batch_size=128, epochs=1, validation_data=(x_test, y_test), )

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 2.3 LSTM

LSTM은 "Long Short-Term Memory"의 약자로, RNN의 변형 중 하나이다. LSTM은 RNN의 단점인 장기 의존성 문제를 해결하고자 고안되었다. 장기 의존성 문제란, RNN이 긴 시퀀스에서 이전 정보를 적절하게 기억하지 못하는 문제를 의미한다.

LSTM은 이 문제를 해결하기 위해 게이트 매커니즘을 도입했는데, 입력 게이트, 삭제 게이트, 출력 게이트 등의 게이트를 사용하여 이전 정보를 얼마나 기억하고 얼마나 새로운 정보를 반영할지를 조절하게 된다. 이를 통해 LSTM은 긴 시퀀스에서도 장기적인 의존성을 학습할 수 있다.

LSTM은 감성 분석과 같은 NLP 작업에 유용하게 사용된다. 오냐하면 LSTM이 시퀀스 데이터를 처리하고 문맥 정보를 유지하면서 텍스트의 감정 표현을 이해하는 데 도움이 되기 때문입니다.


In [ ]:
def build_LSTM_model():
    sequences = layers.Input(shape=(maxlen,))
    embedded = layers.Embedding(max_features, 64)(sequences)
    x = layers.LSTM(128, return_sequences=True)(embedded)
    x = layers.LSTM(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model2 = models.Model(inputs=sequences, outputs=predictions)
    model2.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model2

lstm_model = build_LSTM_model()
 
    


In [ ]:
history_lstm=lstm_model.fit(train_texts, train_labels,  batch_size=128, epochs=1, validation_data=(x_test, y_test), )

In [ ]:
import matplotlib.image  as mpimg

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history_lstm.history['accuracy']
val_acc=history_lstm.history['val_accuracy']
loss=history_lstm.history['loss']
val_loss=history_lstm.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()

## 2.4 CNN+LSTM


CNN과 LSTM을 결합한 모델은 CNN-LSTM 모델이라고 불리며, 이미지나 텍스트와 같은 시퀀스 데이터에 대해 효과적인 모델이다. 이 모델은 CNN 레이어를 통해 특징 추출을 수행하고, 추출된 특징을 LSTM 레이어로 전달하여 시퀀스적인 특징을 학습하여 시간적인 의존성과 지역적인 구조 모두를 이해할 수 있다. CNN-LSTM 모델은 자연어 처리, 영상 분류, 행동 인식 등 다양한 작업에 활용되고 있다.

In [16]:
def cudnnlstm_model(conv_layers = 2, max_dilation_rate = 3):
    inp = layers.Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Conv1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = layers.Conv1D(128*2**(strides), strides = strides, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_size=3, kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = layers.CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = layers.CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x = concatenate([x_f, x_b])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

    return model

cudnnlstm_model = cudnnlstm_model()
cudnnlstm_model.summary()

NameError: name 'Input' is not defined

In [ ]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks = [checkpoint, early_stopping]

In [ ]:
cudnnlstm_model.fit(X_train, train_labels, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.20, callbacks=callbacks)

In [ ]:
cudnnlstm_model.load_weights(weight_path)
score, acc = cudnnlstm_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

## 3. Conclusion

keras기반의 다양한 모델을 통해 텍스트 분석을 실시 해보았다. 결과는 순서로 좋았음을 알수 있다.
